![Clarify Logo](https://global-uploads.webflow.com/5e81e464dad44d3a9a32d1f4/5ed10fc3f1ff8467f4466786_logo.svg)

# Introduction
This notebook start from the point that you can get credentials, authentication token, and read data directly from Clarify, and then proceed to show how to perform forecasting and writting the result back into Clarify.

I added a sentencen here.

## Prerequistes 
* [Clarify](https://www.clarify.io) Account
* Introductory tutorial [LINK]
* `clarify-credentials.json` uploaded to this environment.
* `pyclarify` module


## Getting started
We will be using the PyClarify SDK for authentication, reading and writing signals to the Clarify app. The SDK is mirroring the Clarify API, thus [the reference document](https://docs.clarify.io/reference) will be a good resource if you come across any issues or want to see the capabilities of the API. 

In [ ]:
# install dependencies
!pip install pyclarify

In [ ]:
from pyclarify import APIClient
client = APIClient("./clarify-credentials-7.json")

### Reading items
We will split reading items into two parts:
* Reading the meta data information of your singals
* Reading the data of your signals

Due note that this can be done in a single request, however for this turorial we split them to simplify.

#### Reading Meta data
Your items contain information about all sorts of stuff. This can be location of the item, the engineering unit it displays, the sample interval and so forth. You can actually [create your own labels](https://docs.clarify.io/reference#signal) and add whatever you want to keep your items neat and organised. We will explore that further in the (ref)Writing items section. For now we follow the approach in the reference and create an empty request.

In [ ]:
from pyclarify.models.requests import ItemSelect
empty_request = {
  "items": {
    "include": True, 
  }, 
  "times": {
  }, 
  "series": {
  }
}
meta_data_params = ItemSelect(**empty_request)

In [ ]:
response = client.select_items(meta_data_params)
signal_dict = response.result.items
for signal, meta_data in signal_dict.items():
  print(f"ID: {signal} \t Name: {meta_data.name}")

Here you can see the name and ID of the items your `clarify-credentials.json` have access to. If you were expecting other items you may want to download a different credentials file. Here are the complete list of attribute your first signal contains.

In [ ]:
for value in list(signal_dict.values())[-1]:
  print(value)

#### Reading data


In [ ]:
signal_id = "YOUR SIGNAL ID"

reading_data_request = {
  "items": {
    "include": True,
    "filter": {
      "id": {
        "$in": [
          signal_id
        ]
      }
    }
  },
  "times": {
    "notBefore": "2021-03-13T01:00:00Z" #starting from 
  },
  "series": {
    "items": True,
    "aggregates": False
  }
}

data_params = ItemSelect(**reading_data_request)

In [ ]:
response = client.select_items(data_params)
signal_name = list(response.result.items.values())[0].name
times = response.result.data.times
series = response.result.data.series

In [ ]:
import pandas as pd
df = pd.DataFrame(series)
df.index = times
df.head(5)

In [ ]:
df.plot(figsize=(20, 10));

### Writing data
Writing data is done in two step:
* Writing meta data
* Writing values

Writing meta data can be done with creating a `Signal` data type and populating it with meta data. The ID of this signal needs to correspond with the ID we use for writing value to it.

#### Creating a new Signal
We will do a simple rolling window to create a new Item in the Clarify app. First we want to create the meta data for the Item with a `Signal` data type.

In [ ]:
from pyclarify import Signal
new_signal_name = f"{signal_name}_rolling_mean"
new_signal_id = f"{signal_id}_rolling_mean"


new_signal_meta_data = Signal(
    name=new_signal_name,
    description=f"Rolling window with 1d resolution of the signal {signal_id}",
    labels={
        "rolling_window": ["1 day"],
        "aggregated": [True],
        "aggregated_from": [signal_id]
    },
)

In [ ]:
response = client.save_signals(
    inputs={new_signal_id : new_signal_meta_data},
    created_only=False #False = create new signal, True = update existing signal
)

In [ ]:
response

TODO: Whats with the new id in the SaveResult return

#### Adding data to the new Signal
We use the data we got from the read signal data step and do a simple rolling average with a 1 day interval. Then, we will write these values to the newly created signal.

In [ ]:
df_rolling_mean = df.rolling('1d').mean()
df_rolling_mean.columns=[new_signal_id]

In [ ]:
merged_df = df.join(df_rolling_mean)

In [ ]:
merged_df[signal_id].plot(figsize=(20, 10))
merged_df[new_signal_id].plot(secondary_y=True,figsize=(20, 10));

We use data frames (not to be confused with pandas data frames) to send values to and from Clarify. [Data frames](https://docs.clarify.io/reference#data-frame-1) separates time and values by having the same timestamps for all signal even though they might not have a value at a given timestamp. The backend handles this by not writing null values to Clarify. The signals is a dictionary consiting of signal_id as keys and values as values. 

In [ ]:
from pyclarify import DataFrame
times = merged_df.index.values.tolist()
series = merged_df.to_dict(orient="list")
new_df = DataFrame(times=times, series=series)

In [ ]:
response = client.insert(new_df)

In [ ]:
response

SOME CONCLUSION


NEXT STEPS